In [ ]:
class MyLineReg():
  def __init__(self, n_iter = 100, learning_rate = 0.1, weights = None, metric = None, reg = None, l1_coef = 0, l2_coef = 0, sgd_sample = None, random_state = 42, **kwargs):
    self.n_iter = n_iter
    self.learning_rate = learning_rate
    self.params = kwargs
    self.weights = weights
    self.metric = metric
    self.reg = reg
    self.l1_coef= l1_coef
    self.l2_coef = l2_coef
    self.sgd_sample = sgd_sample
    self.random_state = random_state


  def __str__(self):
    base_str = f'MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'
    if self.params:
      param_str = ', '.join([f"{key}={value}" for key, value in self.params.items()])
      return f"{base_str}, {param_str}"
    return base_str


  def calculate_metric(self, y_true, y_pred):
    if self.metric == 'mae':
      return np.mean(np.abs(y_true - y_pred))
    elif self.metric == 'rmse':
      return np.sqrt(np.mean(np.square(y_true - y_pred)))
    elif self.metric == 'r2':
      return 1 - np.sum(np.square(y_true - y_pred)) / np.sum(np.square(y_true - np.mean(y_true)))
    elif self.metric == 'mape':
      return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    elif self.metric == 'mse':
      return np.mean(np.square(y_true - y_pred))
    else:
      return None


  def fit(self, X, y, verbose = False):
    random.seed(self.random_state)
    X.insert(0, 'Ones', 1)
    self.weights = np.ones(X.shape[1])

    for i in range(self.n_iter):
      if self.sgd_sample:
        if type(self.sgd_sample) == int:
          sample_rows_idx = random.sample(range(X.shape[0]), self.sgd_sample)
        elif type(self.sgd_sample) == float:
          sample_rows_idx = random.sample(range(X.shape[0]), int(self.sgd_sample * X.shape[0]))

        y_pred = X.dot(self.weights)
        X_mini_batch = X.iloc[sample_rows_idx]
        y_mini_batch = y.iloc[sample_rows_idx]
        y_pred_mini = X_mini_batch.dot(self.weights)
        error_mini = y_pred_mini - y_mini_batch
        gradient_mini = 2 / len(y_mini_batch) * error_mini.dot(X_mini_batch)
        MSE = np.mean(np.square(y_pred - y))

        if self.reg == 'l1':
          MSE = MSE + self.l1_coef * np.sum(np.abs(self.weights))
          gradient_mini = gradient_mini + self.l1_coef * np.sign(self.weights)
        elif self.reg == 'l2':
          MSE = MSE + self.l2_coef * np.sum(np.square(self.weights))
          gradient_mini = gradient_mini + self.l2_coef * 2 * self.weights
        elif self.reg == 'elasticnet':
          MSE = MSE + self.l1_coef * np.sum(np.abs(self.weights)) + self.l2_coef * np.sum(np.square(self.weights))
          gradient_mini = gradient_mini + self.l1_coef * np.sign(self.weights) + self.l2_coef * 2 * self.weights

        if isinstance(self.learning_rate, (int, float)):
          self.weights -= self.learning_rate * gradient_mini
        else:
          self.weights -= self.learning_rate(i + 1) * gradient_mini

      else:
        y_pred = X.dot(self.weights)
        error = y_pred - y
        MSE = np.mean(np.square(error))
        gradient = 2 / len(y) * error.dot(X)

        if self.reg == 'l1':
          MSE = MSE + self.l1_coef * np.sum(np.abs(self.weights))
          gradient = gradient + self.l1_coef * np.sign(self.weights)
        elif self.reg == 'l2':
          MSE = MSE + self.l2_coef * np.sum(np.square(self.weights))
          gradient = gradient + self.l2_coef * 2 * self.weights
        elif self.reg == 'elasticnet':
          MSE = MSE + self.l1_coef * np.sum(np.abs(self.weights)) + self.l2_coef * np.sum(np.square(self.weights))
          gradient = gradient + self.l1_coef * np.sign(self.weights) + self.l2_coef * 2 * self.weights

        if isinstance(self.learning_rate, (int, float)):
          self.weights -= self.learning_rate * gradient
        else:
          self.weights -= self.learning_rate(i + 1) * gradient

        self.metric_value = self.calculate_metric(y, y_pred)

      if verbose and (i + 1) % verbose == 0:
        print(f'{i + 1} / loss : {MSE} / {self.metric} : {self.metric_value} / learning_rate : {self.learning_rate}')


  def get_coef(self):
    return np.array(self.weights)[1:]

  def predict(self, X):
    X.insert(0, 'Ones', 1)
    y_pred = X.dot(self.weights)
    return y_pred


  def get_best_score(self):
    return self.metric_value
